<a href="https://colab.research.google.com/github/lucianosilva-github/visaocomputacional/blob/master/Aula%2020/Visa%CC%83o_Computacional_Aula_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **VISÃO COMPUTACIONAL - AULA 20**

**Objetivos da aula:**

*   apresentar o módulo Open3D
*   realizar leitura e visualização de nuvens de pontos
*   conhecer e praticar com o algoritmo BPA para reconstrução de malhas (meshes) para nuvens de pontos


## **FUNDAMENTOS DO MÓDULO OPEN3D**

O módulo Open3D é um módulo C++/Python que suporta o desenvolvimento de aplicações para dados 3D. Ele disponibiliza um conjunto de estruturas de dados e algoritmos, tanto na linguagem C++ quando em Python, inclusive com configurações para paralelização (multicore e manycore). 

A documentação da versão 0.13 pode ser encontrada no seguinte link: 

http://www.open3d.org/docs/release/

O módulo Open3D Python não faz parte das distribuições convencionais do Python e pode ser instalado via pip:

In [ ]:
!pip install open3d

É comum identificarmos o módulo como o3d. A tarefa mais básica em Open3D é ler uma nuvem de pontos. Existem diversos formatos suportados como os formatos PLY e PCD. É comum, para se ganhar eficiência em alguns procedimentos, converter a nuvem de pontos para um vetor de pontos em numpy.

In [ ]:
import open3d as o3d
import numpy as np

pcd = o3d.io.read_point_cloud("points.ply")
print(pcd)
points=np.asarray(pcd.points)
print(points)


O módulo Open3D não tem um suporte muito efetivo para visualização 3D em notebooks Python. Para isto, precisamos de um módulo auxiliar chamado plotply: 

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=1, color=np.asarray(pcd.colors))
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

## **EXERCÍCIO**

Leia e visualize a nuvem de pontos bunny.ply:

In [ ]:
#digite sua solução aqui

## **ESTIMAÇÃO DE MALHA (MESH) DE UMA NUVEM DE PONTOS**

Existem diversos algoritmos para reconstruir malhas a partir de nuvens de pontos. Um dos mais simples e utilizados é o algoritmo BPA (Ball-Pivoting Algorithm): 

*F. Bernardini and J. Mittleman and HRushmeier and C. Silva and G. Taubin: The ball-pivoting algorithm for surface reconstruction, IEEE transactions on visualization and computer graphics, 5(4), 349-359, 1999.*

Link para o artigo: https://vgc.poly.edu/~csilva/papers/tvcg99.pdf 


## **EXERCÍCIO**

Ler e explicar como funciona o algoritmo BPA.

In [ ]:
#explique como funciona o algoritmo BPA aqui

Conforme visto no artigo acima, o algoritmo depende de 4 (quatro) raios, especificados pela lista de raios radii:


In [26]:
pcd = o3d.io.read_point_cloud("bunny.ply")
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
radii = [0.005, 0.01, 0.02, 0.04]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radii))

A partir da malha estimada, estimamos também as normais dos vértices e dos triângulos que compõem a malha:

In [22]:
if not mesh.has_vertex_normals(): 
  mesh.compute_vertex_normals()
if not mesh.has_triangle_normals(): 
  mesh.compute_triangle_normals()

Se a malha não possui cores em suas faces, adicionamos cores artificiais:

In [23]:
triangles = np.asarray(mesh.triangles)
vertices = np.asarray(mesh.vertices)
colors = None
if mesh.has_triangle_normals():
    colors = (0.5, 0.5, 0.5) + np.asarray(mesh.triangle_normals) * 0.5
    colors = tuple(map(tuple, colors))
else:
    colors = (1.0, 0.0, 0.0)

Finalmente podemos desenhar a malha:

In [ ]:
fig = go.Figure(
    data=[
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=triangles[:,0],
            j=triangles[:,1],
            k=triangles[:,2],
            facecolor=colors,
            opacity=0.50)
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

## **EXERCÍCIO**

Variar os raios dos quatro círculos e observar a qualidade da reconstrução da malha.

In [ ]:
#faça seus experimentos aqui

## **EXERCÍCIO**

Reconstrua a malha da nuvem de pontos points.ply, com diferentes raios do algoritmo BPA.

In [ ]:
#implementa sua solução aqui